In [1]:
import numpy as np
import pandas as pd
from tree import decisionTreeNode
import random
import time

In [2]:
def myc(x=0):
    y = 2*(np.log(x-1) + 0.5772156649) -2*(x-1)/x
    return(y)

In [3]:
def nodeInfo(X, Length=0, isLeaf=False, splitAtt=None, splitValue=None, isLeftChild=None, isRoot=True, Amount=0):
    node = decisionTreeNode()
    node.setLength(Length)
    node.setsplitAtt(splitAtt)
    node.setsplitValue(splitValue)
    node.setisLeftChild(isLeftChild)
    node.setisRoot(isRoot)
    node.setAmount(Amount)
    if(isLeaf):
        node.setisLeaf()
    return(node)

In [4]:
def iTree(X, e, l, Length=0, isLeaf=False, splitAtt=None, splitValue=None, isLeftChild=None, isRoot=True, Amount=0):
    if(e < l and len(X.index) > 1):
        node = nodeInfo(X, Length, isLeaf, splitAtt, splitValue, isLeftChild, isRoot, Amount)
        splitAtt = X.columns[random.randint(0, (len(X.columns)-1))]
        minimum = min(X[splitAtt])
        maximum = max(X[splitAtt])
        while(minimum == maximum):
            splitAtt = X.columns[random.randint(0, (len(X.columns)-1))]
            minimum = min(X[splitAtt])
            maximum = max(X[splitAtt])
        splitValue = np.random.uniform(low=minimum, high=maximum)
        Xl = X.loc[X[splitAtt]<=splitValue, :]
        Xr = X.loc[X[splitAtt]>splitValue, :]
        ee = e + 1
        Amountl = len(Xl.index)
        Child_left = iTree(X=Xl, e=ee, l=l, Length=ee, splitAtt=splitAtt, splitValue=splitValue, 
                           isLeftChild=True, isRoot=False, Amount=Amountl)
        node.setChildren(Child_left)
        Amountr = len(Xr.index)
        Child_right = iTree(X=Xr, e=ee, l=l, Length=ee, splitAtt=splitAtt, splitValue=splitValue, 
                            isLeftChild=False, isRoot=False, Amount=Amountr)
        node.setChildren(Child_right)
    else:
        Amount = len(X.index)
        node = nodeInfo(X, Length, isLeaf=True, splitAtt=splitAtt, splitValue=splitValue, 
                        isLeftChild=isLeftChild, isRoot=isRoot, Amount=Amount)
    return(node)

In [5]:
def printTree(node, depth = 0):
    if not node.getisRoot():
        x = node.getsplitAtt()
        if node.getisLeftChild():
            equality = "<="
        else:
            equality = ">"
        threshold = "%.6f" % node.getsplitValue() # Six decimal
        if node.getisLeaf():
            print(depth * "|\t" + "%s %s %s [%d]: %s" \
                                  % (node.getsplitAtt(), equality, threshold,
                                     node.getAmount(), node.getLength()))
        else:
            print(depth * "|\t" + "%s %s %s [%d]" \
                                  % (node.getsplitAtt(), equality, threshold,
                                     node.getAmount()))
        depth +=1
    for child in node.getChildren():
        printTree(child, depth)

In [43]:
def PathLength(X, decisionTree, l):
    prediction = None
    if decisionTree.getisLeaf():
        if decisionTree.getAmount()>1:
            return(decisionTree.getLength() + myc(decisionTree.getAmount()))
        else:
            return(decisionTree.getLength())
    for child in decisionTree.getChildren():
        splitAtt = child.getsplitAtt()
        splitValue = child.getsplitValue()
        if child.getisLeftChild():
            if X[splitAtt] <= splitValue:
                prediction = PathLength(X, child, l)
        else:
            if X[splitAtt] > splitValue:
                prediction = PathLength(X, child, l)
    return(prediction)

In [44]:
def AnomalyScore(x, n):
    return(2**(-x/myc(n)))

In [96]:
# def classify(X, decisionTree):
#     prediction = None
#     if decisionTree.getisLeaf():
#         return(decisionTree.getLength() + myc(decisionTree.getAmount()))
#     for child in decisionTree.getChildren():
#         splitAtt = child.getsplitAtt()
#         splitValue = child.getsplitValue()
#         if child.getisLeftChild():
#             if X[splitAtt] <= splitValue:
#                 prediction = classify(X, child)
#         else:
#             if X[splitAtt] > splitValue:
#                 prediction = classify(X, child)
#     return(prediction)

In [6]:
# def predict(train, test, decisionTree):
#     numData = len(train.index)
#     prediction = np.zeros(numData)
#     for i in range(numData):
#         instance = train.loc[i, :]
#         prediction[i] = classify(instance, decisionTree)
#     pred_y = classify(test, decisionTree)
#     return prediction, pred_y

In [41]:
def predict(dat, decisionTree, l):
    numData = len(dat.index)
    prediction = np.zeros(numData)
    for i in range(numData):
        instance = dat.iloc[i, :]
        prediction[i] = PathLength(instance, decisionTree, l)
    return prediction

In [29]:
# dat = pd.read_csv('../data/dat1.csv')
# FF = np.where(dat.Class==1)
# Row1 = np.arange(1000)
# Row = np.append(Row1, FF[0][2])
# dat_test = dat.loc[Row, :]
# X = dat_test.drop('Class', axis=1)
# e=0
# l=np.log2(len(X.index))
# myTree = iTree(X, e, l)
# printTree(myTree)
# classify(X=dat_test.loc[8,], decisionTree=myTree, l=l)

In [55]:
dat = pd.read_csv('../data/dat1.csv')

In [81]:
t1 = time.time()
phi = 256
N = 11000
dat_use = dat.iloc[range(N), ]
the_list = range(N)
random.seed(1234)
cishu = 100
prediction = np.zeros(N)
for i in range(cishu):
    print(i, end="")
    train_index = random.sample(the_list, phi)
    dat_train = dat_use.iloc[train_index, ]
    X = dat_train.drop('Class', axis=1)
    y = dat_train.Class
    e=0
    l=np.log2(len(X.index))
    myTree = iTree(X, e, l)
    prediction += predict(dat_use, myTree, l)
prediction_final = prediction/cishu

t2 = time.time() 
t2 - t1

0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899

360.80669808387756

In [82]:
result = AnomalyScore(prediction_final, phi)
result

array([0.40414239, 0.37312089, 0.48809006, ..., 0.41981242, 0.37913399,
       0.43975248])

In [86]:
np.where(result>0.55)[0]

array([   85,   140,   146,   164,   174,   225,   290,   362,   370,
         401,   455,   601,   865,  1086,  1158,  1281,  1388,  1608,
        1632,  1722,  1830,  1856,  2016,  2059,  2178,  2182,  2186,
        2234,  2277,  2453,  2457,  2464,  2622,  2629,  2677,  2682,
        2792,  2897,  2951,  2954,  2956,  2957,  2963,  3443,  3509,
        3547,  3554,  3602,  3771,  4235,  4608,  4740,  4776,  4810,
        4895,  4898,  4926,  5257,  5296,  5371,  5425,  5504,  5526,
        5534,  5535,  5652,  5715,  5791,  5797,  5827,  5888,  5916,
        6095,  6108,  6223,  6300,  6305,  6306,  6329,  6330,  6331,
        6334,  6336,  6338,  6410,  6411,  6412,  6413,  6427,  6446,
        6462,  6472,  6501,  6529,  6549,  6609,  6624,  6633,  6641,
        6692,  6717,  6719,  6734,  6774,  6783,  6812,  6820,  6870,
        6882,  6899,  6903,  6905,  6940,  6971,  7029,  7080,  7081,
        7090,  7100,  7103,  7104,  7144,  7255,  7277,  7389,  7420,
        7469,  7485,

In [85]:
np.where(dat_use.Class==1)[0]

array([  541,   623,  4920,  6108,  6329,  6331,  6334,  6336,  6338,
        6427,  6446,  6472,  6529,  6609,  6641,  6717,  6719,  6734,
        6774,  6820,  6870,  6882,  6899,  6903,  6971,  8296,  8312,
        8335,  8615,  8617,  8842,  8845,  8972,  9035,  9179,  9252,
        9487,  9509, 10204, 10484, 10497, 10498, 10568, 10630, 10690,
       10801, 10891, 10897])

In [161]:
t1 = time.time()
N = 50000
dat_train = dat.loc[:N, ]
X = dat_train.drop('Class', axis=1)
y = dat_train.Class
e=0
l=np.log2(len(X.index))
myTree = iTree(X, e, l)
prediction = predict(dat_train, myTree, l)
t2 = time.time() 
t2 - t1

29.19114398956299

In [166]:
np.where(prediction>0.15)[0]

array([  164,   472,   480,   541,   568,   723,  1261,  1480,  1624,
        1632,  2178,  2951,  2954,  2957,  2963,  3099,  3285,  3330,
        3811,  3828,  4840,  4895,  4932,  5322,  5425,  5729,  6057,
        6329,  6331,  6334,  6336,  6338,  6506,  6529,  6609,  6641,
        6684,  6717,  6719,  6743,  6783,  6812,  6820,  6870,  6899,
        6903,  6971,  7100,  7469,  7617,  8311,  8610,  9950, 10201,
       10232, 10630, 10690, 11258, 12832, 12851, 13616, 13768, 13796,
       13876, 15274, 15386, 16301, 17746, 18570, 18633, 18907, 19760,
       19943, 19962, 19985, 20260, 20417, 20418, 20419, 20420, 20829,
       22031, 23128, 23288, 23876, 24160, 24163, 25302, 26060, 27101,
       27648, 29947, 30551, 31799, 32359, 32949, 33846, 34077, 34658,
       35001, 35766, 37000, 38798, 39769, 39973, 40700, 41780, 42294,
       42512, 43116, 43159, 43372, 44223, 44282, 44432, 44513, 44947,
       45446, 45757, 45774, 46497, 46505, 46790, 46841, 46909, 47743,
       48011, 48529,

In [160]:
np.where(dat.Class==1)[0]

array([   541,    623,   4920,   6108,   6329,   6331,   6334,   6336,
         6338,   6427,   6446,   6472,   6529,   6609,   6641,   6717,
         6719,   6734,   6774,   6820,   6870,   6882,   6899,   6903,
         6971,   8296,   8312,   8335,   8615,   8617,   8842,   8845,
         8972,   9035,   9179,   9252,   9487,   9509,  10204,  10484,
        10497,  10498,  10568,  10630,  10690,  10801,  10891,  10897,
        11343,  11710,  11841,  11880,  12070,  12108,  12261,  12369,
        14104,  14170,  14197,  14211,  14338,  15166,  15204,  15225,
        15451,  15476,  15506,  15539,  15566,  15736,  15751,  15781,
        15810,  16415,  16780,  16863,  17317,  17366,  17407,  17453,
        17480,  18466,  18472,  18773,  18809,  20198,  23308,  23422,
        26802,  27362,  27627,  27738,  27749,  29687,  30100,  30314,
        30384,  30398,  30442,  30473,  30496,  31002,  33276,  39183,
        40085,  40525,  41395,  41569,  41943,  42007,  42009,  42473,
      

In [146]:
t1 = time.time()

prediction = np.zeros(n_end-n_start)
for ii in range(n_start, n_end) :
    dat_train = dat.loc[:ii, ]
    dat_test = dat.loc[ii+1, ]
    X = dat_train.drop('Class', axis=1)
    e=0
    l=np.log2(len(X.index))
    myTree = iTree(X, e, l)
    prediction[ii-n_start] = classify(dat_test, myTree, l)
t2 = time.time() 
t2 - t1

2.2440760135650635

In [147]:
np.where(prediction>0.5)[0] + n_start + 1

array([], dtype=int64)

In [148]:
prediction

array([0.25466866, 0.05664236, 0.07448308, 0.17244307, 0.01750793,
       0.03723531, 0.17259744, 0.17264867, 0.17269978, 0.08382089,
       0.02718494, 0.13800413, 0.02158873, 0.02854046, 0.06602584,
       0.02687576, 0.21034936, 0.01874854, 0.02231995, 0.09062555])

In [140]:
np.where(dat.Class==1)[0]
# np.where(dat.Class==1)[0]-n_start-1

array([   541,    623,   4920,   6108,   6329,   6331,   6334,   6336,
         6338,   6427,   6446,   6472,   6529,   6609,   6641,   6717,
         6719,   6734,   6774,   6820,   6870,   6882,   6899,   6903,
         6971,   8296,   8312,   8335,   8615,   8617,   8842,   8845,
         8972,   9035,   9179,   9252,   9487,   9509,  10204,  10484,
        10497,  10498,  10568,  10630,  10690,  10801,  10891,  10897,
        11343,  11710,  11841,  11880,  12070,  12108,  12261,  12369,
        14104,  14170,  14197,  14211,  14338,  15166,  15204,  15225,
        15451,  15476,  15506,  15539,  15566,  15736,  15751,  15781,
        15810,  16415,  16780,  16863,  17317,  17366,  17407,  17453,
        17480,  18466,  18472,  18773,  18809,  20198,  23308,  23422,
        26802,  27362,  27627,  27738,  27749,  29687,  30100,  30314,
        30384,  30398,  30442,  30473,  30496,  31002,  33276,  39183,
        40085,  40525,  41395,  41569,  41943,  42007,  42009,  42473,
      

In [133]:
x = np.array([1,2,3])

In [134]:
np.where(x==2)

(array([1]),)